In [1]:
# Link: https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/LLMReranker-Gatsby.html
from dotenv import load_dotenv

import openai
import os
load_dotenv()

import logging
import sys
import os
import nest_asyncio

import numexpr as ne

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
openai.api_key = os.getenv('OPENAI_API_KEY')
nest_asyncio.apply()


os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'

In [2]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
)
from llama_index.postprocessor import LLMRerank
from llama_index.llms import OpenAI
from IPython.display import Markdown, display

In [30]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

In [31]:
# load documents
documents = SimpleDirectoryReader("../test_data").load_data()

In [32]:
index = VectorStoreIndex.from_documents(
    documents, 
    service_context=service_context
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.open

In [33]:
from llama_index.prompts import PromptTemplate
openai.log = "debug"

text_qa_template_str = (
    "Bạn là trợ lý của Andrew Huberman, người có thể đọc các ghi chú podcast của Andrew Huberman.\n"
    "Luôn trả lời truy vấn chỉ bằng cách sử dụng thông tin ngữ cảnh được cung cấp, "
    "chứ không phải kiến ​​thức sẵn có.\n"
    "Một số quy tắc cần tuân theo:\n"
    "1. Không bao giờ tham khảo trực tiếp bối cảnh nhất định trong câu trả lời của bạn.\n"
    "2. Tránh những câu như 'Dựa trên ngữ cảnh, ...' hoặc "
    "'Thông tin ngữ cảnh ...' hoặc bất cứ điều gì cùng"
    "những dòng đó."
    "Thông tin bối cảnh dưới đây.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Trả lời câu hỏi:{query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [34]:
from llama_index.postprocessor import SentenceTransformerRerank


rerank = SentenceTransformerRerank(
    model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", 
    top_n=3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from time import time

query_engine = index.as_query_engine(
    similarity_top_k=5,
    # node_postprocessors=[rerank],
    text_qa_template = text_qa_template
)

In [36]:
now = time()
response = query_engine.query(
    "Xuất tinh sớm có nguy hiểm hay không?",
)
print(f"Elapsed: {round(time() - now, 2)}s")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Elapsed: 5.07s


In [37]:
print(response)

Xuất tinh sớm không gây nguy hiểm đến tính mạng, nhưng nó có thể ảnh hưởng đến đời sống tình dục, tâm lý và khả năng sinh sản của người đàn ông. Nếu không được điều trị sớm, xuất tinh sớm có thể gây ra những biến chứng nguy hiểm như rối loạn cương dương, gây liệt dương và thậm chí dẫn đến vô sinh.


In [38]:
for i in response.source_nodes:
    print('score: ', i.score)
    print('text: ', i.text)
    print('='*100)

score:  0.8658662541328387
text:  (1) Xuất tinh sớm là vấn đề này khá nhạy cảm nên nhiều người không dám đối diện, né tránh thăm khám, chữa trị. Tuy nhiên, điều này ảnh hưởng rất lớn đến cuộc sống của nam giới, nếu không được điều trị sớm, nam giới có nguy cơ đối mặt với những biến chứng nguy hiểm như: rối loạn cương dương, gây liệt dương, thậm chí dẫn đến vô sinh. Xuất tinh sớm thường có 4 dạng là nguyên phát (thời gian xuất tinh ngắn xảy ra ngay từ những lần đầu giao hợp và kéo dài cho tới hiện tại), thứ phát (thời gian xuất tinh bình thường, nhưng thời gian gây đây có giảm đáng kể ảnh và ảnh hưởng tới tâm lý và quan hệ cặp đôi), xuất tinh sớm thay đổi (thời gian xuất tinh thay đổi tùy thuộc vào hoàn cảnh, đối tác, kích thích…) và các rối loạn xuất tinh giống <strong>xuất tinh sớm</strong>.
score:  0.8657671753421374
text:  Quá trình xuất tinh là sự phóng tinh dịch ra khỏi dương vật khi đạt được cực khoái. Việc xuất tinh sớm không chỉ ảnh hưởng đến của cảm xúc giường chiếu của chính 